# SKLearn Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to deploy iris classifier based on Scikit Learn model using MLP 

In [1]:
!cat requirements.txt

scikit-learn>=1.1.2
caraml
joblib>=0.13.0,<1.2.0  # >=1.2.0 upon upgrade of kserve's version
cloudpickle==2.0.0

In [8]:
!pip list | grep caram

caraml                        0.0.0.post58.dev0
caraml-auth-google            0.0.0.post7
caraml-upi-protos             1.0.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip3 install --upgrade -r requirements.txt > /dev/null


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [11]:
# import basic
import warnings
import os

# Import modeling lib
from sklearn import svm
from sklearn import datasets
from joblib import dump
from sklearn.datasets import load_iris

# Load merlin SDK
import caraml
from 
from models.model import ModelType
# warnings.filterwarnings('ignore')

/Users/kuppilpawanmaniteja/Documents/github/caraml-sdk/venv-caraml-sdk/lib/python3.8/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/kuppilpawanmaniteja/Documents/github/caraml-sdk/venv-caraml-sdk/lib/python3.8/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_endpoint_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/kuppilpawanmaniteja/Documents/github/caraml-sdk/venv-caraml-sdk/lib/python3.8/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_prediction_output" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn

## 1. Initialize MLP Resources

### 1.1 Set MLP Server

In [ ]:
# Set MLP Server
CARAML_URL = os.environ.get("CARAML_URL", "http://console.d.ai.golabs.io")
caraml = 
caram.set_url(CARAML_URL)

In [ ]:
# Print MLP Server URL
print("Merlin Server URL: {}".format(MERLIN_SERVER_URL))

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in MLP is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by MLP are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("sklearn-sample", ModelType.SKLEARN)

## 2. Train and Deploy

### 2.1 Train and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()
```

In [ ]:
model_dir = "sklearn-model"
MODEL_FILE = "model.joblib"

# Create new version of the model
with merlin.new_model_version() as v:

    # Start Model Training
    clf = svm.SVC(gamma='scale')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    clf.fit(X, y)
    
    # Save model to local directory
    dump(clf, os.path.join(model_dir, MODEL_FILE))
    
    # Upload the serialized model to MLP
    merlin.log_model(model_dir=model_dir)

## 2.2 Deploy Model

In [ ]:
endpoint = merlin.deploy(v)

## 2.3 Send Test Request

In [ ]:
%%bash -s "$endpoint.url"
curl -v -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

## 3.4 Delete Deployment

In [ ]:
merlin.undeploy(v)